# Day 19: Monster Messages
---
Ampun, _**Regular Expression**_ bikin pusing.

In [1]:
my_input = None
with open("input.txt") as file:
    my_input = file.read()

In [2]:
def parse_input(input_str):
    rules, messages = input_str.split("\n\n")
    rules = {k:v for k, v in (line.split(": ") for line in rules.strip().split("\n"))}
    messages = [line for line in messages.strip().split("\n")]
    return rules, messages

---
## Part 1
Bagian pertama kita ubah semua *rule* menjadi satu *pattern string* yang akan kita gunakan untuk _**Regular Expression**_.

In [3]:
import re

def part1(input_str):
    rules, messages = parse_input(input_str)

    def get_rules_pattern(rule_id, rules, memo = dict()):
        if not rule_id in memo:
            rule = rules[rule_id].split(" ")
            rule = "".join(map(lambda x: get_rules_pattern(x, rules, memo) if x in rules else x.strip('"'), rule))
            if "|" in rule:
                rule = "(?:"+rule+")"
            memo[rule_id] = rule
        return memo[rule_id]
    
    pattern = get_rules_pattern("0", rules)
    re_patt = re.compile(pattern)
    return len([message for message in messages if re_patt.fullmatch(message)])

In [4]:
%time part1(my_input)

Wall time: 15 ms


291

---
## Part 2
Perubahan aturan *8* cukup menambahkan ```+``` pada akhiran untuk menandakan pola tersebut berulang 1 kali atau lebih.

Sedangkan untuk aturan *11* menandakan polanya rekursif di dalam pola itu sendiri. Modul bawaan python tidak mendukung pola rekursif.

Alternatifnya kita loop sendiri jumlah perulangannya dengan ```while``` dan cari hingga pola tidak menemukan pesan yang valid lagi.

In [5]:

def part2(input_str):
    rules, messages = parse_input(input_str)

    def get_rules_pattern(rule_id, rules, memo = dict()):
        if not rule_id in memo:
            rule = rules[rule_id].split(" ")
            rule = "".join(map(lambda x: get_rules_pattern(x, rules, memo) if x in rules else x.strip('"'), rule))
            if "|" in rule:
                rule = "(?:"+rule+")"
            if rule_id == "8":
                rule = get_rules_pattern("42", rules, memo) + "+"
            if rule_id == "11":
                rule = get_rules_pattern("42", rules, memo) + r"{n}" + get_rules_pattern("31", rules, memo) + r"{n}"
            memo[rule_id] = rule
        return memo[rule_id]
    
    pattern = get_rules_pattern("0", rules)
    prev_result = None
    result = 0
    n = 1
    while result != prev_result:
        prev_result = result
        re_patt = re.compile(pattern.replace("n", str(n)))
        result += len([message for message in messages if re_patt.fullmatch(message)])
        n += 1
    return result

In [6]:
%time part2(my_input)

Wall time: 74 ms


409